# Leave One Complex Out

June 18th, 2022


In [1]:
!dir()

 Volume in drive C has no label.

File Not Found


 Volume Serial Number is 022C-93CC

 Directory of C:\Users\ibrah\Documents\GitHub\Predator\src\Benchmark\LeaveOneComplexOut



Now, we are using only one random forest classifier in leave one out scheme.

In [2]:
from utils import (
    load_protein_dataframes,
    get_top_n_features,
    get_train_and_test_data,
    get_predictions,
    get_scoring_metrics,
    read_features,
    read_unique_proteins,
)

from tqdm.notebook import tqdm

HIDDEN_ENTRIES_FOLDER_PATH = "hidden_entries_2022-06-18"

TOP_N = 10

# Reflect changes in the modules immediately.
%load_ext autoreload
%autoreload 2

In [4]:
unique_proteins = read_unique_proteins("unique_proteins_ordered_2022-06-18.txt")
len(unique_proteins)

164

In [5]:
protein_dataframes = load_protein_dataframes(HIDDEN_ENTRIES_FOLDER_PATH, order_list=unique_proteins)
len(protein_dataframes)

164

In [6]:
unique_proteins[56]

'P56945'

In [7]:
protein_dataframes[56]

,Distinguished_Index,Mutation_Effect_Label,Input_identifier,UniProt_ID,Mutation,Status,Type,COSMIC_mut_ID,ClinVar_mut_ID,UniProt_mut_ID,...,IntraclashesEnergy2_wt,IntraclashesEnergy2_mut,Interface_hydrophobic_area,Interface_hydrophilic_area,Interface_total_area,Interface_contact_distance_wt,Interface_contact_distance_mut,Template_cath_id_pdb,Chain_id,Interactor_template_cath_id_pdb
0,Entry_00130,1,P56945,P56945,L787E,done,interface,-,-,-,...,81.2478,81.2418,494.040,245.74,739.780,3.87320,3.85202,3t6g,B,3t6g
1,Entry_00386,1,P56945,P56945,F794R,done,interface,-,-,-,...,47.5017,47.5154,128.975,91.13,220.105,3.38758,3.10238,3t6g,B,3t6g
2,Entry_00387,1,P56945,P56945,F794R,done,interface,-,-,-,...,40.5543,40.5543,129.175,92.44,221.605,3.52071,3.45489,3t6g,B,3t6g
3,Entry_00388,1,P56945,P56945,F794R,done,interface,-,-,-,...,40.5543,40.5543,129.175,92.44,221.605,3.52071,3.45489,3t6g,B,3t6g
4,Entry_00389,1,P56945,P56945,F794R,done,interface,-,-,-,...,35.9280,35.9394,128.655,79.47,208.125,3.36360,3.45580,3t6g,B,3t6g
5,Entry_00390,1,P56945,P56945,F794R,done,interface,-,-,-,...,78.5007,78.3554,494.040,245.74,739.780,3.46975,2.77228,3t6g,B,3t6g
6,Entry_00391,1,P56945,P56945,F794R,done,interface,-,-,-,...,35.9280,35.9394,128.655,79.47,208.125,3.36360,3.45580,3t6g,B,3t6g
7,Entry_00617,1,P56945,P56945,D797R,done,interface,-,-,-,...,78.5007,79.3309,494.040,245.74,739.780,2.59894,2.99905,3t6g,B,3t6g
8,Entry_00688,1,P56945,P56945,F794E,done,interface,-,-,-,...,47.5017,47.5154,128.975,91.13,220.105,3.38758,3.08500,3t6g,B,3t6g
9,Entry_00689,1,P56945,P56945,F794E,done,interface,-,-,-,...,40.5543,40.5543,129.175,92.44,221.605,3.46028,3.13051,3t6g,B,3t6g


In [8]:
len(read_features("features.txt"))

58

In [9]:
import pandas as pd

In [10]:
def run_leave_one_complex_out(protein_dfs, ordered_unique_proteins, features=read_features("features.txt")):

    selected_features_list = []
    prediction_dataframes = []

    for index, protein in tqdm(
            enumerate(ordered_unique_proteins), total=len(ordered_unique_proteins)
    ):
        # excluding test data
        train_data, test_data = get_train_and_test_data(protein_dfs, index=index)
        current_pred_data = test_data.copy()

        print("Current prediction data: ")
        display(current_pred_data)

        X_train = train_data[features].apply(pd.to_numeric, errors="coerce").fillna(0)
        y_train = train_data["Mutation_Effect_Label"]

        X_test = test_data[features].apply(pd.to_numeric, errors="coerce").fillna(0)
        y_test = test_data["Mutation_Effect_Label"]

        current_pred_data["Actual"] = y_test

        n_train = len(X_train)
        n_test = len(X_test)
        print(f"Training RF model with {n_train} entries.")
        print(f"Predicting {n_test} entries. The protein: {test_data['UniProt_ID'].unique()}")

        # Predictions with all features and no hyper tuning
        current_pred_data["Prediction_default"] = get_predictions(X_train, y_train, X_test, tuning=False)

        # Feature selection
        selected_features = get_top_n_features(X_train, y_train, top_n=TOP_N)
        selected_features_list.append(selected_features)

        # Update X_train and X_test with selected features
        X_train_fs = X_train[selected_features]
        X_test_fs = X_test[selected_features]

        # Predictions with selected features and no hyper tuning
        current_pred_data["Prediction_FS"] = get_predictions(
            X_train_fs, y_train, X_test_fs, tuning=False
        )

        # Predictions with selected features and hyper tuning
        current_pred_data["Prediction_FS_HP"] = get_predictions(
            X_train_fs, y_train, X_test_fs, tuning=True
        )


        prediction_dataframes.append(current_pred_data)


        print("=======================================")


    return {
        "prediction_dataframes": prediction_dataframes,
        "selected_features_list": selected_features_list,
    }


In [11]:
out = run_leave_one_complex_out(
    protein_dfs=protein_dataframes, ordered_unique_proteins=unique_proteins
)

  0%|          | 0/164 [00:00<?, ?it/s]

Current prediction data: 


,Distinguished_Index,Mutation_Effect_Label,Input_identifier,UniProt_ID,Mutation,Status,Type,COSMIC_mut_ID,ClinVar_mut_ID,UniProt_mut_ID,...,IntraclashesEnergy2_wt,IntraclashesEnergy2_mut,Interface_hydrophobic_area,Interface_hydrophilic_area,Interface_total_area,Interface_contact_distance_wt,Interface_contact_distance_mut,Template_cath_id_pdb,Chain_id,Interactor_template_cath_id_pdb
0,Entry_00000,1,Q9BPZ3,Q9BPZ3,F118A,done,interface,-,-,-,...,52.4675,52.4675,427.775,194.715,622.495,3.35293,3.35293,1jgn,B,1jgn


Training RF model with 739 entries.
Predicting 1 entries. The protein: ['Q9BPZ3']
selected_features=['EL2_score', 'Provean_score', 'Model/DOPE_score', 'Final_ddG', 'Matrix_score', 'entropy_mainchain_wt', 'Solvent_accessibility_mut', 'entropy_mainchain_mut', 'ProtBert_score', 'sidechain_hbond_mut']
Current prediction data: 


,Distinguished_Index,Mutation_Effect_Label,Input_identifier,UniProt_ID,Mutation,Status,Type,COSMIC_mut_ID,ClinVar_mut_ID,UniProt_mut_ID,...,IntraclashesEnergy2_wt,IntraclashesEnergy2_mut,Interface_hydrophobic_area,Interface_hydrophilic_area,Interface_total_area,Interface_contact_distance_wt,Interface_contact_distance_mut,Template_cath_id_pdb,Chain_id,Interactor_template_cath_id_pdb
0,Entry_00001,1,P01116,P01116,Y40C,done,interface,-,-,-,...,70.2902,70.2461,464.47,400.03,864.505,3.07827,3.59123,3ddc,A,3ddc
1,Entry_00657,1,P01116,P01116,E37G,done,interface,-,-,-,...,70.2449,70.2449,464.47,400.03,864.505,2.63614,3.21798,3ddc,A,3ddc


...

Training RF model with 738 entries.
Predicting 2 entries. The protein: ['P19429']
selected_features=['Provean_score', 'EL2_score', 'Model/DOPE_score', 'Final_ddG', 'Matrix_score', 'entropy_mainchain_wt', 'sidechain_hbond_mut', 'ProtBert_score', 'entropy_mainchain_mut', 'Solvent_accessibility_mut']
Current prediction data: 


,Distinguished_Index,Mutation_Effect_Label,Input_identifier,UniProt_ID,Mutation,Status,Type,COSMIC_mut_ID,ClinVar_mut_ID,UniProt_mut_ID,...,IntraclashesEnergy2_wt,IntraclashesEnergy2_mut,Interface_hydrophobic_area,Interface_hydrophilic_area,Interface_total_area,Interface_contact_distance_wt,Interface_contact_distance_mut,Template_cath_id_pdb,Chain_id,Interactor_template_cath_id_pdb
0,Entry_00711,0,P21860,P21860,G284R,done,interface,COSM941488,-,-,...,68.9744,68.9744,None,None,None,3.69640,3.69640,3u7u,A,3u7u
1,Entry_00712,0,P21860,P21860,G284R,done,interface,COSM941488,-,-,...,45.8003,45.8003,716.79,358.84,1075.62,4.33509,4.33509,3u7u,A,3u7u


Training RF model with 738 entries.
Predicting 2 entries. The protein: ['P21860']
selected_features=['Provean_score', 'EL2_score', 'Model/DOPE_score', 'Matrix_score', 'Final_ddG', 'entropy_mainchain_wt', 'sidechain_hbond_mut', 'ProtBert_score', 'entropy_mainchain_mut', 'Solvent_accessibility_mut']
Current prediction data: 


,Distinguished_Index,Mutation_Effect_Label,Input_identifier,UniProt_ID,Mutation,Status,Type,COSMIC_mut_ID,ClinVar_mut_ID,UniProt_mut_ID,...,IntraclashesEnergy2_wt,IntraclashesEnergy2_mut,Interface_hydrophobic_area,Interface_hydrophilic_area,Interface_total_area,Interface_contact_distance_wt,Interface_contact_distance_mut,Template_cath_id_pdb,Chain_id,Interactor_template_cath_id_pdb
0,Entry_00730,1,P30622-2,P30622-2,K252A,done,interface,-,-,-,...,15.5605,15.5605,298.115,118.055,416.175,3.59716,4.22529,1txq,A,1txq


Training RF model with 739 entries.
Predicting 1 entries. The protein: ['P30622-2']
selected_features=['Provean_score', 'EL2_score', 'Model/DOPE_score', 'Matrix_score', 'Final_ddG', 'entropy_mainchain_wt', 'sidechain_hbond_mut', 'entropy_mainchain_mut', 'Solvent_accessibility_mut', 'ProtBert_score']


In [12]:
leave_one_complex_out_result_data = pd.concat(out["prediction_dataframes"], ignore_index=True)
leave_one_complex_out_result_data.sort_values("Distinguished_Index", ignore_index=True, inplace=True)
leave_one_complex_out_result_data

,Distinguished_Index,Mutation_Effect_Label,Input_identifier,UniProt_ID,Mutation,Status,Type,COSMIC_mut_ID,ClinVar_mut_ID,UniProt_mut_ID,...,Interface_total_area,Interface_contact_distance_wt,Interface_contact_distance_mut,Template_cath_id_pdb,Chain_id,Interactor_template_cath_id_pdb,Actual,Prediction_default,Prediction_FS,Prediction_FS_HP
0,Entry_00000,1,Q9BPZ3,Q9BPZ3,F118A,done,interface,-,-,-,...,622.495,3.35293,3.35293,1jgn,B,1jgn,1,0,0,1
1,Entry_00001,1,P01116,P01116,Y40C,done,interface,-,-,-,...,864.505,3.07827,3.59123,3ddc,A,3ddc,1,1,1,1
2,Entry_00002,1,Q96QK1,Q96QK1,F534D,done,interface,-,-,-,...,809.51,3.03938,3.24435,2r17,C,2r17,1,1,1,1
3,Entry_00003,1,Q96QK1,Q96QK1,F534D,done,interface,-,-,-,...,809.51,3.03938,3.24435,2r17,C,2r17,1,1,1,1
4,Entry_00004,1,O43521-3,O43521-3,G66A,done,interface,-,-,-,...,679.245,3.37301,3.22505,3fdl,B,3fdl,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,Entry_00735,1,P84243,P84243,Q94A,done,interface,-,-,-,...,2112.45,3.00752,4.27950,4hga,B,4hga,1,1,1,1
736,Entry_00736,1,Q96QK1,Q96QK1,L589D,done,interface,-,-,-,...,809.51,3.45879,3.26826,2r17,C,2r17,1,1,1,1
737,Entry_00737,1,Q96QK1,Q96QK1,L589D,done,interface,-,-,-,...,809.51,3.45879,3.26826,2r17,C,2r17,1,1,1,1
738,Entry_00738,1,P23297,P23297,F72L,done,interface,-,-,-,...,956.72,3.26388,3.30643,2y5i,A,2y5i,1,1,1,1


In [13]:
from datetime import datetime

current_date = datetime.today().strftime('%Y-%m-%d')

In [14]:
leave_one_complex_out_result_data.to_csv(f"leave_one_complex_out_result_data_112_cols_{current_date}.csv", index=False)

In [15]:
selected_columns = [
    "Mutation_Effect_Label",
    "UniProt_ID", "Mutation",
    "Interactor_UniProt_ID",
    "Template_cath_id",
    "Chain_id",
    "Actual",
    "Prediction_default",
    "Prediction_FS",
    "Prediction_FS_HP"
]

leave_one_complex_out_result_data_simplified = leave_one_complex_out_result_data[selected_columns].copy()
leave_one_complex_out_result_data_simplified

,Mutation_Effect_Label,UniProt_ID,Mutation,Interactor_UniProt_ID,Template_cath_id,Chain_id,Actual,Prediction_default,Prediction_FS,Prediction_FS_HP
0,1,Q9BPZ3,F118A,P11940,1jgnB,B,1,0,0,1
1,1,P01116,Y40C,P50749,3ddcA,A,1,1,1,1
2,1,Q96QK1,F534D,Q9UBQ0,2r17C,C,1,1,1,1
3,1,Q96QK1,F534D,Q9UBQ0-2,2r17C,C,1,1,1,1
4,1,O43521-3,G66A,Q07812,3fdlB,B,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
735,1,P84243,Q94A,Q9UER7-3,4hgaB,B,1,1,1,1
736,1,Q96QK1,L589D,Q9UBQ0,2r17C,C,1,1,1,1
737,1,Q96QK1,L589D,Q9UBQ0-2,2r17C,C,1,1,1,1
738,1,P23297,F72L,P25815,2y5iA,A,1,1,1,1


In [16]:
leave_one_complex_out_result_data_simplified["Mutation_Effect_Label"].equals(
    leave_one_complex_out_result_data_simplified["Actual"]
)

True

In [17]:
leave_one_complex_out_result_data_simplified.to_csv(f"leave_one_complex_out_result_data_simplified_{current_date}.csv", index=False)

In [18]:
y_true = leave_one_complex_out_result_data_simplified["Actual"].tolist()
y_pred_default = leave_one_complex_out_result_data_simplified["Prediction_default"].tolist()
y_pred_fs = leave_one_complex_out_result_data_simplified["Prediction_FS"].tolist()
y_pred_fs_hypertuned = leave_one_complex_out_result_data_simplified["Prediction_FS_HP"].tolist()

In [19]:
import pandas as pd

metrics_comparison_data = pd.concat(
    [
        get_scoring_metrics(y_true, y_pred_default),
        get_scoring_metrics(y_true, y_pred_fs),
        get_scoring_metrics(y_true, y_pred_fs_hypertuned)
    ], axis="columns"
)
metrics_comparison_data.columns = ["Default", "Feature selected", "Feature selected + hypertuned"]
metrics_comparison_data

,Default,Feature selected,Feature selected + hypertuned
ACCURACY,0.781081,0.760811,0.747297
BALANCED_ACCURACY,0.670556,0.648796,0.612778
F1,0.858639,0.844873,0.839485
MATTEWS_COR,0.393386,0.338368,0.276488
PRECISION,0.811881,0.801997,0.782400
RECALL,0.911111,0.892593,0.905556


# ---------------------------------------------------------------

In [ ]:
# The end.